In [123]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ViTModel
from torch.nn.functional import softmax
import pandas as pd 
import os
import matplotlib.cm as cm
from PIL import Image
import numpy as np

In [7]:
df = pd.read_csv("../train.csv")

In [8]:
df.head()

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


In [62]:
cmap = cm.get_cmap("viridis")

/tmp/ipykernel_46409/4067785149.py:1: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap("viridis")


In [102]:
class spectrogramDataset(Dataset):
    def __init__(self,csv_file,path="../train_spectrograms/",transform=None):
        self.df = pd.read_csv(csv_file)
        self.transform = transform
        self.path = path
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self,idx):
        spec_id = df.iloc[idx,:]['spectrogram_id']
        label = df.iloc[idx,:]['expert_consensus']
        spectrogram_path = os.path.join(self.path,str(spec_id)+".parquet")
        # print(spectrogram_path)
        spectrogram = pd.read_parquet(spectrogram_path).drop('time',axis=1).values
        # print(spectrogram)
        spectrogram = Image.fromarray((cmap(spectrogram) * 255).astype(np.uint8))
        
        if self.transform:
            spectrogram = self.transform(spectrogram)[:3, :, :]
        
        
        return spectrogram,label
        
        

In [94]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
#     transforms.CenterCrop(224),  # Center crop to maintain aspect ratio
    transforms.ToTensor(),  # Convert to PyTorch tensor
#     transforms.Normalize(mean=[0.485, 0.456, 0.406, 0.406], std=[0.229, 0.224, 0.225, 0.225])  # Normalize (optional)
])

In [107]:
dataset = spectrogramDataset("../train.csv",transform=transform)

In [108]:
dataset.__len__()

106800

In [110]:
dataset.__getitem__(10000)

(tensor([[[0.9922, 0.9922, 0.9922,  ..., 0.1804, 0.1647, 0.1686],
          [0.9922, 0.9922, 0.9922,  ..., 0.1490, 0.1804, 0.2196],
          [0.9922, 0.9922, 0.9922,  ..., 0.1765, 0.1922, 0.2235],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0.9059, 0.9059, 0.9059,  ..., 0.5059, 0.4980, 0.4706],
          [0.9059, 0.9059, 0.9059,  ..., 0.5176, 0.4314, 0.3451],
          [0.9059, 0.9059, 0.9059,  ..., 0.4471, 0.4078, 0.3294],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0.1412, 0.1412, 0.1412,  ..., 0.5373, 0.5451, 0.5490],
          [0.1412, 0.1412, 0.1412,  ..., 0.5451, 0.5490, 0.5490],
          [0.1412, 0.1412, 0.1412,  ...,

In [113]:
train_len = int(.8*len(dataset))
val_len = len(dataset)-train_len

In [114]:
val_len

21360

In [116]:
train_set,val_set = torch.utils.data.random_split(dataset,[train_len,val_len])

In [120]:
train_loader = DataLoader(train_set,batch_size=32,shuffle=True,num_workers=4,prefetch_factor=1)
val_loader = DataLoader(val_set, batch_size=32,shuffle=True,num_workers=4,prefetch_factor=1)

In [124]:
class ViTClassifier(torch.nn.Module):
    def __init__(self,num_classes=5):
        super().__init__()
        self.vit = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")
        self.classifer = torch.nn.Linear(self.vit.config.hidden_size,num_classes)
        
    def forward(self,images):
        output = self.vit(images)
        output = self.classifier(output.last_hidden_state[:,0])
        output = softmax(output,dim=1)
        return output

In [126]:
model = ViTClassifier(num_classes=6)

In [127]:
loss_fn = torch.nn.KLDivLoss(reduction='batchmean')
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [9]:
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k')


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
device

device(type='cpu')